[View in Colaboratory](https://colab.research.google.com/github/Yozh2/ClocksOrCrocs/blob/master/Clock_or_Crocodile.ipynb)

# Распознавание часов и крокодилов на изображениях с помощью предварительно обученной нейронной сети VGG16

**Источник данных** - [Архив с изображениями](https://drive.google.com/file/d/1JbYmH50iRkMorFk0xNCnwC9xKiB60Mlq/view?usp=sharing), полученный по почте.

Для распознавания используется предварительно обученная сверточная нейронная сеть VGG16.


In [0]:
import os
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.applications import VGG16
from tensorflow.python.keras.optimizers import Adam


Перед использованием данных, их необходимо разбить на обучающую, проверочную и тестовую выборки. Делается это при помощи скрипта `data_preparation`

In [0]:
# Каталог с данными для обучения
train_dir = './train'
# Каталог с данными для проверки
val_dir = './val'
# Каталог с данными для тестирования
test_dir = './test'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Размер мини-выборки
batch_size = 64
# Количество изображений для обучения
nb_train_samples = 350
# Количество изображений для проверки
nb_validation_samples = 75
# Количество изображений для тестирования
nb_test_samples = 75

Подкачиваем файлы с датасетом из [GitHub](https://github.com/Yozh2/ClocksOrCrocs/tree/master/data)

In [77]:
import os
git_dir = '/content/ClocksOrCrocs'
os.chdir(git_dir)
!git pull

# Переносимся в папку data
os.chdir(os.path.join(git_dir, 'data'))

Already up-to-date.


## Загружаем предварительно обученную нейронную сеть

In [0]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

"Замораживаем" веса предварительно обученной нейронной сети VGG16

In [0]:
vgg16_net.trainable = False

In [70]:
vgg16_net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

## Создаем составную нейронную сеть на основе VGG16

In [0]:
model = Sequential()
# Добавляем в модель сеть VGG16 вместо слоя
model.add(vgg16_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [72]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               2097408   
_________________________________________________________________
activation_3 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
_________________________________________________________________
activation_4 (Activation)    (None, 1)                 0         
Total para

Компилируем составную нейронную сеть

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])

## Создаем генератор изображений

Генератор изображений создается на основе класса ImageDataGenerator. Генератор делит значения всех пикселов изображения на 255.

In [0]:
datagen = ImageDataGenerator(rescale=1. / 255)


Генератор данных для обучения на основе изображений из каталога

In [78]:
!ls 

test  train  val


In [79]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 700 images belonging to 2 classes.


Генератор данных для проверки на основе изображений из каталога

In [80]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 150 images belonging to 2 classes.


Генератор данных для тестирования на основе изображений из каталога

In [81]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 150 images belonging to 2 classes.


## Обучаем модель с использованием генераторов

train_generator - генератор данных для обучения

validation_data - генератор данных для проверки

In [82]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
5/5 [==============================] - 95s 19s/step - loss: 0.8045 - acc: 0.4813 - val_loss: 0.6651 - val_acc: 0.6406
Epoch 2/10
5/5 [==============================] - 90s 18s/step - loss: 0.6939 - acc: 0.5800 - val_loss: 0.6147 - val_acc: 0.7188
Epoch 3/10
5/5 [==============================] - 81s 16s/step - loss: 0.6492 - acc: 0.6250 - val_loss: 0.5420 - val_acc: 0.7727
Epoch 4/10
5/5 [==============================] - 90s 18s/step - loss: 0.6395 - acc: 0.6461 - val_loss: 0.5560 - val_acc: 0.8438
Epoch 5/10
5/5 [==============================] - 90s 18s/step - loss: 0.6044 - acc: 0.6743 - val_loss: 0.4888 - val_acc: 0.9375
Epoch 6/10
5/5 [==============================] - 82s 16s/step - loss: 0.5515 - acc: 0.7344 - val_loss: 0.4391 - val_acc: 0.9091
Epoch 7/10
5/5 [==============================] - 90s 18s/step - loss: 0.5222 - acc: 0.7465 - val_loss: 0.4464 - val_acc: 0.8906
Epoch 8/10
5/5 [==============================] - 91s 18s/step - loss: 0.5005 - acc: 0.7656 - val

## Оцениваем качество работы сети с помощью генератора

In [0]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

In [84]:
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

Аккуратность на тестовых данных: 87.50%


# Тонкая настройка сети (fine tuning)

"Размораживаем" последний сверточный блок сети VGG16

In [0]:
vgg16_net.trainable = True
trainable = False
for layer in vgg16_net.layers:
    if layer.name == 'block5_conv1':
        trainable = True
    layer.trainable = trainable    

In [86]:
# Проверяем количество обучаемых параметров
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               2097408   
_________________________________________________________________
activation_3 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
_________________________________________________________________
activation_4 (Activation)    (None, 1)                 0         
Total para

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])

In [88]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=2,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/2
5/5 [==============================] - 105s 21s/step - loss: 0.4070 - acc: 0.8312 - val_loss: 0.3167 - val_acc: 0.9219
Epoch 2/2
5/5 [==============================] - 104s 21s/step - loss: 0.3108 - acc: 0.8986 - val_loss: 0.2385 - val_acc: 0.9062


In [89]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

Аккуратность на тестовых данных: 92.19%
